In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as scisig
import scipy.optimize as opt
import scipy.io as sio
import simulation_utils as su
import analysis_utils as au

In [ ]:
importlib.reload(au)

# Load in noise and response templates

filename1 ='D:/Simulation Data/240529/Frequency modulation/'

yfb_l = np.array([10])*2*np.pi
iter_l = [0.0, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2]
for yfb in yfb_l:

    numbins = int(2*5*10**4*100*2*np.pi/yfb)

    filename = 'D:/Simulation Data/240529/Frequency modulation/yfb' + str(round(yfb/2/np.pi)) + '/' + str(iter)

    data = au.load_data_hdf5(filename+'/Noise/PSD.hdf5')
    PSD_freq = data['frequency']
    PSD_noise = data['J']

    data = au.load_data_hdf5(filename+'/Kicks/avr_resp.hdf5')
    avr_resp = data['x']

    lb = 400
    ub = 1000
    plt.figure()
    plt.plot(PSD_freq[lb:ub], PSD_noise[lb:ub])
    plt.plot(PSD_freq[lb:ub], au.Linewidth2(np.array(PSD_freq[lb:ub]), 0.3*10**(-12), 55000, 1000, 10**(-28)))
    plt.yscale('log')

    plt.figure()
    plt.plot(avr_resp)

    phi_t = au.make_optimal_filter_ns(avr_resp, PSD_noise, PSD_freq)

    plt.plot(phi_t*10**(-9))

    corr_max = []
    #corr_arg = []
    #corr_data_l = np.zeros((1000, numbins))
    for i in range(1000):
        data = au.load_data_hdf5(filename+'/Kicks/' + str(i) +'.hdf5')
        x = data['x']
        cm = au.optimal_filter(phi_t, x)
        corr_max.append(cm)
    print(1)
    
    corr_max_n = []
    #corr_data_l_n = np.zeros((1000, numbins))
    np.random.seed(3716)
    for i in range(1000):
        data = au.load_data_hdf5(filename+'/Noise/' + str(i) +'.hdf5')
        x = data['x']
        cm = au.optimal_filter_noise(phi_t, x)
        corr_max_n.append(cm)
    print(2)
    cf = 55000
    BW = 30000
    amp_max = []
    for i in range(1000):
        max = au.bandpass_peak_find2(filename+'/Kicks/' + str(i) +'.hdf5', cf, BW, 10**6, 3)
        amp_max.append(max)
    
    print(3)
    
    amp_max_n = []
    for i in range(1000):
        max = au.bandpass_peak_find_noise2(filename+'/Noise/' + str(i) +'.hdf5', cf, BW, 10**6, 3)
        amp_max_n.append(max)

    print(4)
    
    analysis_dict = {'Kicks': corr_max, 'Kicks noise': corr_max_n, 'Band-pass filter': amp_max, 'Band-pass filter noise': amp_max_n}
    au.save_data_hdf5(filename1 + 'analysis' + str(int(yfb/2/np.pi)) + '_' + str(iter) +'.hdf5', analysis_dict)